In [15]:
import pysam
import os
import glob
import operator
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from IPython.display import clear_output
import vcf
import csv
import yaml

In [32]:
config_file_name = "miseq_consensus.config.yaml"
config = yaml.load(open(config_file_name))

od=config["out_dir"]
alignment_dir = os.path.join(config["out_dir"], "alignment")

out_dir = os.path.join(config["out_dir"], "consensuses")
!mkdir -p $od $alignment_dir $out_dir

consensus_amb_resolved_file = os.path.join(out_dir, "consensus_amb_resolved.fas")
amb_thrs =[.05, 0.1, 0.2]
consensus_with_amb_files = [os.path.join(out_dir, "consensus_with_amb_{}.fas".format(str(int(x*100)))) for x in amb_thrs]

amb_dict = {
    "A":"A",
    "C":"C",
    "G":"G",
    "T":"T",
    "N":"N",
    "AC":"M",
    "AG":"R",
    "AT":"W",
    "CG":"S",
    "CT":"Y",
    "GT":"K",
    "ACG":"V",
    "ACT":"H",
    "AGT":"D",
    "CGT":"B",
    "ACGT":"N"
}

1) Align sequences

In [ ]:
c='"{}"'.format(alignment_dir)
!rm -f $c/*

if config['is_pair_ended']:
    sample_ids = set()
    for f in glob.glob(os.path.join(config['fastq_dir'], "*")):
        sample_ids.add(os.path.basename(f).replace("_R1_001.fastq","").replace("_R2_001.fastq", ''))
    for s in sample_ids:
        ref = config['ref']
        r1 = os.path.join(config['fastq_dir'], s + '_R1_001.fastq')
        r2 = os.path.join(config['fastq_dir'], s + '_R2_001.fastq')
        o = os.path.join(alignment_dir, s + '.sam')
        !bwa mem -t 8 $ref $r1 $r2 > $o
else:
    for f in glob.glob(os.path.join(config['fastq_dir'], "*")):
        ref = config['ref']
        o = os.path.join(alignment_dir, os.path.splitext(os.path.basename(f))[0] + '.sam')
        !bwa mem -t 8 $ref $f > $o
    

fs = glob.glob(os.path.join(alignment_dir,"*.sam"))
i=0
for f in fs:
    clear_output(wait=True)
    i+=1
    print("{}/{}".format(i,len(fs)))
    b = os.path.splitext(f)[0]
    o = b + ".bam"
    obs = b + "_sorted.bam"
    c='-S -b "{}" > "{}"'.format(f,o)
    !samtools view $c
    c='"{}" -o "{}"'.format(o,obs)
    !samtools sort $c
    c='"{}"'.format(obs)
    !samtools index $c

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 89062 sequences (26718600 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 42391, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (541, 541, 541)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (541, 541)
[M::mem_pestat] mean and std.dev: (541.00, 0.00)
[M::mem_pestat] low and high boundaries for proper pairs: (541, 541)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 89062 reads in 65.144 CPU sec, 8.235 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -t 8 ref/ref.fas raw_reads/8639_S82_L001_R1_001.fastq raw_reads/8639_S82_L001_R2_001.fastq
[main] Real time: 48.926 sec; CPU: 66.293 sec
[M::bwa_idx_load_from_disk] r

2) Extract consensuses

In [8]:
ofs = [consensus_amb_resolved_file] + consensus_with_amb_files
otr = [1] + amb_thrs
seqs=[list() for _ in range(len(ofs))]

def get_key(nucls_dict, thr):
    cons = max(nucls_dict.iteritems(), key=operator.itemgetter(1))[0]
    total = float(sum(nucls_dict.values()))
    if not total:
        return "N"
    filtered_nucl = [k for k,v in nucls_dict.iteritems() if v/total >= thr]
    return "".join(sorted(list(set([cons] + filtered_nucl))))

fs=glob.glob(os.path.join(alignment_dir,"*_sorted.bam"))
cc=0
for f in fs:
    clear_output(wait=True)
    cc+=1
    print("{}/{}".format(cc,len(fs)))
    seq_id = re.sub("_sorted.bam$","",os.path.basename(f))
    try:
        samfile = pysam.AlignmentFile(f, "rb")
    except:
        continue
    consensus = list()
    for c in samfile.pileup(max_depth=10e6):
        consensus.append({"A":0,"C":0,"T":0,"G":0})
        for r in c.pileups:
            if not r.is_del and not r.is_refskip:
                consensus[-1][r.alignment.query_sequence[r.query_position]] += 1
    for i in range(len(ofs)):
        seq = "".join([amb_dict[get_key(x,otr[i])]
                       for x in consensus])
        seqs[i].append(SeqRecord(Seq(seq),id=seq_id,description=seq_id))
for i in range(len(ofs)):
    SeqIO.write(seqs[i],ofs[i], "fasta")

25/25
